In [1]:
from utils import *
import pandas as pd

def format_data(input_df):
    columns = ['Date','Temporada', 'SeasonStage', 'Season Type', 'MetaEquipo', 'Opponent', 'Venue', 'GF', 'GA', 'xG', 'xGA', 'Result', 'Jornada', 'current_points', 'ranking',
            'current_points_home', 'current_points_away', 'current_wins', 'current_wins_home', 'current_wins_away'
            , 'current_draws', 'current_draws_home', 'current_draws_away'
                ,  'current_losses', 'current_losses_home', 'current_losses_away', 
                'partidos_jugados', 'partidos_jugados_home', 'partidos_jugados_away' ]


    data = input_df[~input_df.Jornada.isna()][columns].copy()
    data = data[data.SeasonStage.isin(['Apertura', 'Clausura'])]
    data['SeasonTag'] = data.apply(lambda row: row["Temporada"] + "-" + row['SeasonStage'], axis=1)
    data['GF>0'] = data.apply(lambda row: 1 if row["GF"] > 0 else 0, axis=1)
    data['GA>0'] = data.apply(lambda row: 1 if row["GA"] > 0 else 0, axis=1)
    data['AA'] = data.apply(lambda row: 1 if (row["GF"] > 0 and row['GA'] > 0) else 0, axis = 1)
    data['Over'] = data.apply(lambda row: 1 if (row["GF"] + row['GA'] > 2.5) else 0, axis = 1)
    data['Under'] = data.apply(lambda row: 1 if (row["Over"] == 0) else 0, axis = 1)

    data = data.drop(columns=['SeasonStage','Season Type'])

    new_columns = list(data)
    new_columns.remove("SeasonTag")
    new_columns = ["SeasonTag"] + new_columns

    data = data[new_columns]
    return data

def clean_data(input_df):
    data = input_df[~input_df.Result.isna()].copy()
    data = data[~data.SeasonTag.isin(["2024-2025-Apertura", "2019-2020-Clausura"])]
    return data

load_path = "data/csvdata/scores_and_fixtures.csv"
raw_data = pd.read_csv(load_path).iloc[:,1:]

data = format_data(raw_data)
data = clean_data(data)

In [70]:
from scipy.stats import gamma, poisson
import numpy as np

def compute_teams_stats(data):

    teams = data.MetaEquipo.unique().tolist()
    teams_stats = dict(zip(teams, [{} for _ in range(len(teams))]))
    stats_vars = ['current_points', 'current_wins', 'current_draws', 'current_losses', 
                    'GF', 'GA', 'GF>0', 'GA>0', 'Over', 'Under']
    for team in teams:

        df_points_results = data[data.MetaEquipo == team].groupby(['MetaEquipo', 'SeasonTag']).agg({
                                                        'Temporada' : 'max',
                                                        'partidos_jugados' : 'max',
                                                        'current_points' : 'max', #Gaussian (mu, sigma)
                                                        'current_wins' : 'max', #Poisson lambda >= 2 (percentiles)
                                                        'current_draws' : 'max', #Poisson lambda >= 2
                                                        'current_losses' : 'max', #Poisson lambda >= 2
                                                        'GF' : 'sum', #Poisson lambda in [1,2] (percentiles)
                                                        'GA' : 'sum', #Poisson lambda in [1,2]
                                                        'GF>0' : 'sum', #Poisson lambda in [1,2]
                                                        'GA>0' : 'sum', #Poisson lambda in [1,2]
                                                        'Over' : 'sum', #Gaussian
                                                        'Under': 'sum', #Gaussian

                                                        }).reset_index()


        # Estimate Gamma prior parameters
        alpha = 2
        beta = 1

        # Sample from the posterior distribution
        n = len(df_points_results)
        
        for var in stats_vars:            # Number of observations
            sum_x = df_points_results[var].sum()  # Sum of observed values

            alpha_post, beta_post = alpha + sum_x , beta + n 

            num_samples = 10000
            posterior_samples = gamma.rvs(alpha_post, scale=1/beta_post, size=num_samples)

            # Compute the MAP estimate
            hist, bin_edges = np.histogram(posterior_samples, bins=30, density=True)
            bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
            map_estimate = np.ceil(bin_centers[np.argmax(hist)])

            # Posterior Predictive Distribution
            predicted_scores = [poisson.rvs(mu=lmb, size=n) for lmb in posterior_samples]
            predicted_scores = np.array(predicted_scores).flatten()
            if var in ['current_points', 'Over', 'Under']:
                teams_stats[team][var] = {"μ" : np.round(predicted_scores.mean()), "σ" : np.round(predicted_scores.std())}
            elif var in ['current_wins', 'current_draws', 'current_losses']:
                teams_stats[team][var] = {"Percentile 10%" : np.quantile(predicted_scores, 0.1), "Percentile 85%" : np.quantile(predicted_scores, 0.85),
                "100%" : np.quantile(predicted_scores, 1), "λ" : map_estimate}
            else:
                teams_stats[team][var] = {"Percentile 20%" : np.quantile(predicted_scores, 0.2), "Percentile 90%" : np.quantile(predicted_scores, 0.90),
                "100%" : np.quantile(predicted_scores, 1), "λ" : map_estimate}
    
    return teams_stats

teams_stats = compute_teams_stats(data)

In [86]:
questions = [
    '¿Cuántos puntos hace una temporada promedio?', # μ
    '¿Cuántos partidos suele ganar?', # λ
    '¿Cuántos partidos suele perder?', # λ
    '¿Cuántos partidos suele empatar?', # λ
    '¿Cuántos goles anota?', # λ
    '¿Cuántos goles recibe?', # λ
    '¿Cuál es su mejor temporada?', # argmax(current_points)
    '¿Cuál es su mejor año?', # argmax(avg(current_points))
    '¿Su último año ha sido igual al esperado, arriba o abajo?',# μ < avg(current_points)
    '¿Contra qué equipo tiene mejores resultados?', 
    '¿Contra qué equipo tiene peores resultados?',
    '¿Cuál es el equipo que más les cuesta visitar?',
    '¿Cuál es el equipo que más se les facilita visitar?',
    '¿Cuál es el equipo que más les cuesta vencer en casa?',
    '¿Cuál es el equipo que más se les facilita en casa?'
]

teams = data.MetaEquipo.unique().tolist()

report = pd.DataFrame(index=teams, columns=questions)

for team in teams:
    df_points_results = data[data.MetaEquipo == team].groupby(['MetaEquipo', 'SeasonTag']).agg({
                                                            'Temporada' : 'max',
                                                            'partidos_jugados' : 'max',
                                                            'current_points' : 'max', #Gaussian (mu, sigma)
                                                            'current_wins' : 'max', #Poisson lambda >= 2 (percentiles)
                                                            'current_draws' : 'max', #Poisson lambda >= 2
                                                            'current_losses' : 'max', #Poisson lambda >= 2
                                                            'GF' : 'sum', #Poisson lambda in [1,2] (percentiles)
                                                            'GA' : 'sum', #Poisson lambda in [1,2]
                                                            'GF>0' : 'sum', #Poisson lambda in [1,2]
                                                            'GA>0' : 'sum', #Poisson lambda in [1,2]
                                                            'Over' : 'sum', #Gaussian
                                                            'Under': 'sum', #Gaussian

                                                            }).reset_index()

    year_df = df_points_results.groupby(by=['MetaEquipo', 'Temporada']).agg({'current_points' : 'mean'}).reset_index().sort_values(by='Temporada')
    team_opponent_results = {}

    for mode in ['General', 'Home', 'Away']:
        if mode == 'General':
            team_matches_data = data[(data.MetaEquipo == team) & (data.Opponent.isin(inverse_name_mapping.keys()))][['Opponent','Result']]
        elif mode == 'Home':
            team_matches_data = data[(data.MetaEquipo == team) & (data.Opponent.isin(inverse_name_mapping.keys())) & (data.Venue == 'Home')][['Opponent','Result']]
        else:
            team_matches_data = data[(data.MetaEquipo == team) & (data.Opponent.isin(inverse_name_mapping.keys())) & (data.Venue == 'Away')][['Opponent','Result']]

        team_matches_data['win'] = team_matches_data['Result'].apply(lambda result: 1 if result == 'W' else 0)
        team_matches_data['loss'] = team_matches_data['Result'].apply(lambda result: 1 if result == 'L' else 0)
        team_matches_data['draw'] = team_matches_data['Result'].apply(lambda result: 1 if result == 'D' else 0)
        general_rate_data = team_matches_data[['Opponent', 'win', 'loss', 'draw']].groupby('Opponent').sum().reset_index()
        total_data = team_matches_data[['Opponent', 'win', 'loss', 'draw']].groupby('Opponent').count().reset_index()
        general_rate_data[['win','loss','draw']] /= total_data[['win','loss','draw']]
        team_opponent_results[mode] = general_rate_data

    general_opponent_report = team_opponent_results['General']
    away_opponent_report = team_opponent_results['Away']
    home_opponent_report = team_opponent_results['Home']

    report.loc[team,questions[0]] = teams_stats[team]['current_points']['μ']
    report.loc[team,questions[1]] = teams_stats[team]['current_wins']['λ']
    report.loc[team,questions[2]] = teams_stats[team]['current_losses']['λ']
    report.loc[team,questions[3]] = teams_stats[team]['current_draws']['λ']
    report.loc[team,questions[4]] = teams_stats[team]['GF']['λ']
    report.loc[team,questions[5]] = teams_stats[team]['GA']['λ']
    report.loc[team,questions[6]] = df_points_results[df_points_results.current_points == df_points_results.current_points.max()].SeasonTag.values[0]
    report.loc[team,questions[7]] = year_df[year_df.current_points == year_df.current_points.max()].Temporada.values[0]
    report.loc[team,questions[8]] = 'Mejor' if year_df.iloc[-1].current_points > teams_stats[team]['current_points']['μ'] else 'Peor' if year_df.iloc[-1].current_points < teams_stats[team]['current_points']['μ'] else 'Igual'
    report.loc[team,questions[9]] = general_opponent_report[general_opponent_report.win == general_opponent_report.win.max()].Opponent.values
    report.loc[team,questions[10]] = general_opponent_report[general_opponent_report.loss == general_opponent_report.loss.max()].Opponent.values
    report.loc[team,questions[11]] = away_opponent_report[away_opponent_report.loss == away_opponent_report.loss.max()].Opponent.values
    report.loc[team,questions[12]] = away_opponent_report[away_opponent_report.win == away_opponent_report.win.max()].Opponent.values
    report.loc[team,questions[13]] = home_opponent_report[home_opponent_report.loss == home_opponent_report.loss.max()].Opponent.values
    report.loc[team,questions[14]] = home_opponent_report[home_opponent_report.win == home_opponent_report.win.max()].Opponent.values

In [88]:
report.to_csv('ReportePreliminar.csv')

In [87]:
report

,¿Cuántos puntos hace una temporada promedio?,¿Cuántos partidos suele ganar?,¿Cuántos partidos suele perder?,¿Cuántos partidos suele empatar?,¿Cuántos goles anota?,¿Cuántos goles recibe?,¿Cuál es su mejor temporada?,¿Cuál es su mejor año?,"¿Su último año ha sido igual al esperado, arriba o abajo?",¿Contra qué equipo tiene mejores resultados?,¿Contra qué equipo tiene peores resultados?,¿Cuál es el equipo que más les cuesta visitar?,¿Cuál es el equipo que más se les facilita visitar?,¿Cuál es el equipo que más les cuesta vencer en casa?,¿Cuál es el equipo que más se les facilita en casa?
America,29.0,9.0,4.0,5.0,28.0,17.0,2023-2024-Apertura,2023-2024,Mejor,[Atlético],[León],[Monterrey],[Atlético],[Pachuca],[Tijuana]
Atlas,18.0,5.0,8.0,5.0,18.0,23.0,2021-2022-Apertura,2021-2022,Peor,[FC Juárez],[Mazatlán],[Mazatlán],[Querétaro],[Mazatlán],[FC Juárez]
Cruz_Azul,23.0,7.0,6.0,5.0,22.0,21.0,2018-2019-Apertura,2018-2019,Mejor,"[Atlético, FC Juárez]",[América],[Pachuca],[Toluca],[América],[FC Juárez]
Guadalajara,23.0,7.0,6.0,5.0,21.0,19.0,2022-2023-Clausura,2023-2024,Mejor,[FC Juárez],[UANL],[UANL],[FC Juárez],[UANL],[Mazatlán]
Leon,25.0,7.0,6.0,4.0,25.0,22.0,2018-2019-Clausura,2019-2020,Peor,[Necaxa],"[Monterrey, UNAM]",[Pachuca],[Necaxa],"[Monterrey, UNAM]","[Atlético, FC Juárez]"
Monterrey,28.0,9.0,4.0,5.0,28.0,18.0,2022-2023-Clausura,2022-2023,Mejor,"[Atlas, Pachuca]",[Guadalajara],[Toluca],[Mazatlán],[Guadalajara],"[FC Juárez, UNAM]"
Pachuca,25.0,7.0,6.0,5.0,26.0,22.0,2021-2022-Clausura,2022-2023,Mejor,[Puebla],[Monterrey],[Monterrey],"[Atlético, FC Juárez]",[Monterrey],"[León, Tijuana]"
Puebla,19.0,5.0,7.0,5.0,21.0,25.0,2015-2016-Apertura,2021-2022,Peor,[Mazatlán],[Atlético],[Atlético],[FC Juárez],[Necaxa],[Mazatlán]
Pumas_UNAM,21.0,6.0,6.0,5.0,23.0,23.0,2015-2016-Apertura,2015-2016,Mejor,[León],[Monterrey],[Monterrey],[Mazatlán],[Puebla],[FC Juárez]
Queretaro,18.0,5.0,7.0,6.0,18.0,23.0,2019-2020-Apertura,2019-2020,Mejor,[FC Juárez],[Atlas],"[Monterrey, UANL]",[FC Juárez],[Atlas],"[Atlético, FC Juárez]"


In [82]:
team_opponent_results['Away']

,Opponent,win,loss,draw
0,América,0.000000,0.666667,0.333333
1,Atlas,0.666667,0.333333,0.000000
2,Atlético,0.000000,1.000000,0.000000
3,Cruz Azul,0.333333,0.666667,0.000000
4,FC Juárez,0.333333,0.333333,0.333333
5,Guadalajara,0.333333,0.666667,0.000000
6,León,0.333333,0.666667,0.000000
7,Monterrey,0.000000,0.666667,0.333333
8,Necaxa,0.000000,0.666667,0.333333
9,Pachuca,0.000000,0.333333,0.666667
